<a href="https://colab.research.google.com/github/IWNMWE/Song-Recomendation-Chat-Bot-ACM-/blob/main/LSTM_CNN(Emotion_extraction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This is the test notebook for the emotion extraction from text model**

In this note book we aim to create a model to classify text from the twitter data set into 4 emotions joy,sadness,anger,fear


In [39]:
import pandas as pd
data=pd.read_csv("/content/tweets.csv")
data.head()

,Unnamed: 0,Text,label,intensity
0,0,How the fu*k! Who the heck! moved my fridge!....,anger,0.938
1,1,So my Indian Uber driver just called someone ...,anger,0.896
2,2,DPD_UK I asked for my parcel to be delivered ...,anger,0.896
3,3,so ef whichever butt wipe pulled the fire ala...,anger,0.896
4,4,Don't join BTCare they put the phone down on ...,anger,0.896


In [ ]:
for i in range(0,data.shape[0]):
  if (data['label'][i]=='#poem'):
    data['label'][i]='sadness'

In [40]:
data=data.drop(["Unnamed: 0"],axis=1)

In [ ]:
list(data['Text'])

[' How the fu*k! Who the heck! moved my fridge!... should I knock the landlord door.',
 " So my Indian Uber driver just called someone the N word. If I wasn't in a moving vehicle I'd have jumped out",
 ' DPD_UK I asked for my parcel to be delivered to a pick up store not my address',
 ' so ef whichever butt wipe pulled the fire alarm in davis bc I was sound asleep',
 " Don't join BTCare they put the phone down on you, talk over you and are rude. Taking money out of my acc willynilly!",
 ' My blood is',
 " When you've still got a whole season of Wentworth to watch and a stupid cunt in work ruins it for us __KirstyGA",
 " bt_uk why does tracking show my equipment delivered, when it wasn't? Why is my service suddenly delayed? We've already 3 weeks.",
 ' TeamShanny legit why i am so furious with him, people are such fucking',
 ' How is it suppose to work if you do that? Wtf dude? Thanks for pissing me',
 ' im so mad about power rangers. im incensed. im',
 ' Wont use using mothercareuk Moth

In [41]:
data['Text']=data['Text'].str.lower()

In [ ]:
data.to_csv("tweets1.csv")

**These are various ways to preprocess we can explare when we want to create our own tokenizer class**

In [ ]:
from nltk import word_tokenize
data['Text']=data['Text'].apply(lambda X: word_tokenize(str(X)))
data.head()

,Text,label,intensity
0,"[how, the, fu, *, k, !, who, the, heck, !, mov...",anger,0.938
1,"[so, my, indian, uber, driver, just, called, s...",anger,0.896
2,"[dpd_uk, i, asked, for, my, parcel, to, be, de...",anger,0.896
3,"[so, ef, whichever, butt, wipe, pulled, the, f...",anger,0.896
4,"[do, n't, join, btcare, they, put, the, phone,...",anger,0.896


In [ ]:
from textblob import TextBlob
def spell_check(text):
    result = []
    for word in text:
        b = TextBlob(word)
        correct_word = b.correct()
        result.append(str(correct_word))
    
    return result

In [ ]:
data['Text']=data['Text'].apply(spell_check)
data.head()

,Text,label,intensity
0,"[how, the, ff, *, k, !, who, the, neck, !, mov...",anger,0.938
1,"[so, my, indian, user, driver, just, called, s...",anger,0.896
2,"[dpd_uk, i, asked, for, my, parcel, to, be, de...",anger,0.896
3,"[so, of, whichever, butt, wipe, pulled, the, f...",anger,0.896
4,"[do, n't, join, became, they, put, the, phone,...",anger,0.896


In [ ]:
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')

def remove_stopwords(text):
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)
            
    return result

In [ ]:
data['Text']=data['Text'].apply(remove_stopwords)
data.head()

,Text,label,intensity
0,"[ff, *, k, !, neck, !, moved, fridge, !, ..., ...",anger,0.938
1,"[indian, user, driver, called, someone, n, wor...",anger,0.896
2,"[dpd_uk, asked, parcel, delivered, pick, store...",anger,0.896
3,"[whichever, butt, wipe, pulled, fire, alarm, d...",anger,0.896
4,"[n't, join, became, put, phone, ,, talk, rude,...",anger,0.896


In [ ]:
from nltk.tokenize import RegexpTokenizer

def remove_punct(text):
    
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

#Test
text=data['Text'][0] 
print(text) 
remove_punct(text)

['ff', '*', 'k', '!', 'neck', '!', 'moved', 'fridge', '!', '...', 'knock', 'landlord', 'door', '.']


['ff', 'k', 'neck', 'moved', 'fridge', 'knock', 'landlord', 'door']

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

def lemmatization(text):
    
    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
            
        result.append(wordnet.lemmatize(token,pos))
    
    return result

#Test
text = ['running','ran','runs'] 
lemmatization(text)

['run', 'ran', 'run']

In [ ]:
data['Text']=data['Text'].apply(lemmatization)
data.head()

,Text,label,intensity
0,"[ff, k, neck, move, fridge, knock, landlord, d...",anger,0.938
1,"[indian, user, driver, call, someone, n, word,...",anger,0.896
2,"[dpd_uk, ask, parcel, deliver, pick, store, ad...",anger,0.896
3,"[whichever, butt, wipe, pull, fire, alarm, dav...",anger,0.896
4,"[n, t, join, become, put, phone, talk, rude, t...",anger,0.896


**Model Building**

We use a standard LSTM-CNN model to analyse the processed twitter data fro 4 major emotions joy ,fear ,anger and sadness\

The architecture that is known to perform the best on this model was a plainn LSTM and LSTM-CNN model

*references*:

https://konukoii.com/blog/2018/02/19/twitter-sentiment-analysis-using-combined-lstm-cnn-models/

https://www.academia.edu/35947062/Twitter_Sentiment_Analysis_using_combined_LSTM-CNN_Models

In [5]:
import tensorflow as tf

In [6]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer

In [46]:
for i in range(0,data.shape[0]):
  data['Text'][i]=str(data['Text'][i])
  

<ipython-input-46-0347a25b36e5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Text'][i]=str(data['Text'][i])


In [44]:

from pathlib import Path

In [59]:
num_words = 10000

tokenizer = Tokenizer(num_words=num_words, lower=True)
tokenizer.fit_on_texts(list(data.Text))

file_to_save = Path('./tokenizer.pickle').resolve()
with file_to_save.open('wb') as file:
    pickle.dump(tokenizer, file)

In [13]:
from sklearn.model_selection import train_test_split

In [60]:
X_train, X_test, y_train, y_test = train_test_split(list(data['Text']), list(data['label']), test_size=0.20, random_state=42)

In [15]:
from tensorflow.keras.layers import Input, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Conv1D, Dense, concatenate
from tensorflow.keras.models import Model

In [61]:
input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
num_classes = len(data.label.unique())
embedding_dim = 500
input_length = 100
lstm_units = 128
lstm_dropout = 0.1
recurrent_dropout = 0.1
spatial_dropout=0.2
filters=64
kernel_size=3

In [62]:
input_layer = Input(shape=(input_length,))
output_layer = Embedding(
  input_dim=input_dim,
  output_dim=embedding_dim,
  input_shape=(input_length,)
)(input_layer)

output_layer = SpatialDropout1D(spatial_dropout)(output_layer)

output_layer = Bidirectional(
LSTM(lstm_units, return_sequences=True,
     dropout=lstm_dropout, recurrent_dropout=recurrent_dropout)
)(output_layer)
output_layer = Conv1D(filters, kernel_size=kernel_size, padding='valid',
                    kernel_initializer='glorot_uniform')(output_layer)

avg_pool = GlobalAveragePooling1D()(output_layer)
max_pool = GlobalMaxPooling1D()(output_layer)
output_layer = concatenate([avg_pool, max_pool])

output_layer = Dense(num_classes, activation='softmax')(output_layer)

model = Model(input_layer, output_layer)

In [63]:
data['label'].unique()

array(['anger', 'fear', 'joy', 'sadness'], dtype=object)

In [64]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 100, 500)     5000000     ['input_6[0][0]']                
                                                                                                  
 spatial_dropout1d_5 (SpatialDr  (None, 100, 500)    0           ['embedding_5[0][0]']            
 opout1D)                                                                                         
                                                                                                  
 bidirectional_5 (Bidirectional  (None, 100, 256)    644096      ['spatial_dropout1d_5[0][0]

In [53]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer

In [65]:
train_sequences = [text.split() for text in X_train]
validation_sequences = [text.split() for text in X_test]
list_tokenized_train = tokenizer.texts_to_sequences(train_sequences)
list_tokenized_validation = tokenizer.texts_to_sequences(validation_sequences)
x_train = pad_sequences(list_tokenized_train, maxlen=input_length)
x_validation = pad_sequences(list_tokenized_validation, maxlen=input_length)

encoder = LabelBinarizer()
encoder.fit(data.label.unique())

LabelBinarizer()

In [ ]:
y_train = encoder.transform(y_train)
y_validation = encoder.transform(y_test)

In [68]:
batch_size = 32
epochs = 2

In [69]:
model.fit(
    x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_validation, y_validation)
)

Epoch 1/2
178/178 [==============================] - 269s 1s/step - loss: 1.2383 - accuracy: 0.4202 - val_loss: 0.9055 - val_accuracy: 0.6411
Epoch 2/2
178/178 [==============================] - 262s 1s/step - loss: 0.5819 - accuracy: 0.7872 - val_loss: 0.7568 - val_accuracy: 0.7234


In [71]:
model_file = Path('model_weights.h5').resolve()
model.save_weights(model_file.as_posix())